# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [95]:
import pandas as pd
import numpy as np

# Importando os dois dataframes

excel = pd.ExcelFile('tweets_airpods_201809062235.xlsx')
data_treinamento = pd.read_excel(excel, 'Treinamento')
data_teste = pd.read_excel(excel, 'Teste')

# Limpando as colunas Treinamento e Teste dos dataframes

treinamento = data_treinamento['Treinamento'].str.replace('@', '')
treinamento = treinamento.str.replace('rt', '')
treinamento = treinamento.str.replace(',', '')
treinamento = treinamento.str.replace('.', '')
treinamento = treinamento.str.replace(':', '')
treinamento = treinamento.str.replace(';', '')
treinamento = treinamento.str.replace('\n', '')
treinamento = treinamento.str.replace('"', '')

teste = data_teste['Teste'].str.replace('@', '')
teste = teste.str.replace('rt', '')
teste = teste.str.replace(',', '')
teste = teste.str.replace('.', '')
teste = teste.str.replace(':', '')
teste = teste.str.replace(';', '')
teste = teste.str.replace('\n', '')
teste = teste.str.replace('"', '')

# Inserindo as colunas limpas em seus respectivos dataframes

data_treinamento['Treinamento'] = treinamento
data_teste['Teste'] = teste

# Selecionando os relevantes e irrelevantes da base de treinamento

Relevantes = data_treinamento[data_treinamento['PRODUTO : APPLE AIRPODS'] == 'Relevante']
Irrelevantes = data_treinamento[data_treinamento['PRODUTO : APPLE AIRPODS'] == 'Irrelevante']

# Definindo P(Relevante) e P(Irrelevante)

PRelevante = len(Relevantes)/len(data_treinamento)
PIrrelevante = len(Irrelevantes)/len(data_treinamento)

# Comparando P(Relevante|tweet) com P(Irrelevante|tweet)
# P(Relevante|tweet) = P(tweet|Relevante) x P(Relevante)
# P(Irrelevante|tweet) = P(tweet|Irrelevante) x P(Irrelevante)

# Criando as listas com tweets, sendo cada um uma lista com as palavras separadas

lista_tweets = []
lista_tweets_relevantes = []
lista_tweets_irrelevantes = []

for tweet in Relevantes['Treinamento']:
    lista_tweets_relevantes.append(tweet.split())
    
for tweet in Irrelevantes['Treinamento']:
    lista_tweets_irrelevantes.append(tweet.split())

for tweet in treinamento:
    lista_tweets.append(tweet.split())
    
# Verificando palavras possiveis para Laplace Smoothing

palavras_possiveis = []

for tweet in lista_tweets:
    for palavra in tweet:
        if palavra.lower() not in palavras_possiveis:
            palavras_possiveis.append(palavra.lower())
            
# Calcula P(palavra|Relevante) com Laplace Smoothing    
    
def ProbRelevante(palavra):
    ocorrencia = 1
    total_de_palavras = 0
    
    for tweet in lista_tweets_relevantes:
        total_de_palavras += len(tweet)
        
        for i in tweet:
            if i.lower() == palavra.lower():
                ocorrencia += 1
                
    return ocorrencia/(total_de_palavras + len(palavras_possiveis))

# Calcula P(palavra|Irrelevante) com Laplace Smoothing

def ProbIrrelevante(palavra):
    ocorrencia = 1
    total_de_palavras = 0
    
    for tweet in lista_tweets_irrelevantes:
        total_de_palavras += len(tweet)
        
        for i in tweet:
            if i.lower() == palavra.lower():
                ocorrencia += 1
    
    return ocorrencia/(total_de_palavras + len(palavras_possiveis))

# Classificando os tweets da base de treinamento

def classifica(lista_de_tweets):
    classificacao = []

    for tweet in lista_de_tweets: 
        
        tweet_s = tweet.split()
        
        PPRelevantes = []
        PPIrrelevantes = []
                
        for palavra in tweet_s:
                        
            PPRelevantes.append(ProbRelevante(palavra))
            PPIrrelevantes.append(ProbIrrelevante(palavra))

        PTweetRelevante = 1
        PTweetIrrelevante = 1

        for i in PPRelevantes:
            PTweetRelevante *= i

        for i in PPIrrelevantes:
            PTweetIrrelevante *= i

        PRelevanteTweet = PTweetRelevante * PRelevante
        PIrrelevanteTweet = PTweetIrrelevante * PIrrelevante

        if PRelevanteTweet > PIrrelevanteTweet:
            classificacao.append('Relevante')
        else:
            classificacao.append('Irrelevante')
    return classificacao

lista_tweets_teste = []
for tweet in teste:
    lista_tweets_teste.append(tweet)

classificacao = classifica(lista_tweets_teste)

gabarito = data_teste.rename(index=str, columns={'Unnamed: 1':'Relevancia'}).Relevancia

_yontz
airpods
really
the
greatest
pair
of
headphones
i’ve
ever
owned
basspods
custom
made
earpods!
😍⚡️
no
more
tangled
wires
⚡️
wireless
sound
perfected
⚡️
rechargeable
charger
box⚡️
bluetooth
calls…
femi_golden
new
wireless
ale
🤟🏽apple
airpods
✨with
charging
casen67000please
help
🙏🏾
https//tco/oje69gjcp4
i
want
some
airpods
thehighkage_
got
my
airpods!
🔥
thinking
of
passive
crypto
income?
look
no
fuher
than
airpod’s
apod
utility
tokens!
https//tco/mgkvvbvft8
i
just
knew
somebody
in
durham
had
my
airpods
and
i
would
never
see
them
again
😂😂
jayy_dg2f
did
you
get
ya
airpods?
19-year-old
bikes
past
with
airpods
on
shouting
the
words
to
bulls
on
paradewhatyearisitgif
wsj
next
week
apple
is
expected
to
announce
a
new
apple
watch
and
airpods
along
with
a
passel
of
new
iphones
https//tco/qzkbmlo0nl
apple
airpodsany
good?
these
huawei
freebuds
are
cack
thinking
of
passive
crypto
income?
look
no
fuher
than
airpod’s
apod
utility
tokens!
https//tco/s7t7v4fkox
https//tco/a50q9lfmne
are
airpods
wo

_xlavanya
valkyri06801156
i
had
wireless
ones
before
the
airpods
and
these
were
super
sufficient
too
managing
cables
while
working
out
or
jogging
is
annoying
so
that
was
why
i
got
wireless
ones
pierce
this
is
a
really
sma
mims
column
and
i've
been
feeling
this
change
for
a
while
my
favorite
thing
about
owning
an
iphone
is…
don't
want
beatsx
headphones?
try
apple's
airpods
https//tco/2eerrhdnx0
whoever
found
my
airpods
at
the
stephanie
eos
parking
lot
merry
christmas
🎅🏼
airpods
are
literally
the
best
thing
ever
i
can
leave
me
phone
downstairs
and
still
listen
to
music
when
i’m
upstairs
🎉
tfw
i
leave
the
house
for
what
will
be
a
long
walk
and
i
leave
my
airpods
at
home
🤬🤬🤬
y’alli
was
peeing
and
i
looked
down
at
the
urinal
and
one
of
my
airpods
fell
out
circled
the
urinal
and
bounced
outthat
shit
was
full
of
someone’s
pee
omfg
i’m
so
happy
it
didn’t
fall
in😭😭😭
valkyri06801156
_xlavanya
why
doesn’t
he
even
own
airpods
and
make
you
struggle
with
terminology?!
what
a
jerk
thinking
of
passive

some
airpods
21xunqi
please🙏🙏
https//tco/fwehyafowe
hubspot
happy
#inbound18!
we're
on
the
lookout
for
a
winner
of
airpods
today
in
our
#hubspotscavengerhunt!
show
us
some
#hubspotlove
b…
what
to
expect
from
next
week’s
big
iphone
event
#appleevent
9/12/18
two
sizes
—
58
and
65
inches
referred
to
as
the
#iphonexs
and
#iphonexsplus
a
cheaper
#iphonex
the
#applewatch
series
4
#homepodmini
a
new
#macbookair
#airpower
#airpods
with
water
resistance
#ios12
⌚️📱💻
cottonbureau
🚨🚨🚨
flash
giveaway
🚨🚨🚨huge
congrats
to
mkbhd
on
hitting
1
billion
views
on
youtube
to
celebrate
we’re
giving
away
on…
airpods
with
ios
12!
https//tco/skznvka8k5
#airpods
#airpod
#airpods
#ios
#ios12
#iphone
#technology
waiting
to
see
the
first
nigerian
to
use
airpods
in
a
photo
shoot
useful
airpods
tips
&amp
tricks!https//tco/aapnsboy24
https//tco/yk4u331sey


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [99]:
falpos = 0
falneg = 0
verpos = 0
verneg = 0

for i in range(len(gabarito)):
    if gabarito[i] == classificacao[i] and gabarito[i] == 'Relevante':
        verpos += 1
    elif gabarito[i] == classificacao[i] and gabarito[i] == 'Irrelevante':
        verneg += 1
    elif gabarito[i] != classificacao[i] and gabarito[i] == 'Relevante':
        falneg += 1
    else:
        falpos += 1

falpos /= len(classificacao)
falneg /= len(classificacao)
verpos /= len(classificacao)
verneg /= len(classificacao)


print("Negativo Falso: {}%".format(falneg*100))
print("Positivo Falso: {}%".format(falpos*100))
print("Negativo Verdadeiro: {}%".format(verneg*100))
print("Positivo Verdadeiro: {}%".format(verpos*100))

len(classificacao)

Negativo Falso: 5.5%
Positivo Falso: 16.0%
Negativo Verdadeiro: 72.5%
Positivo Verdadeiro: 6.0%


200

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
